# Importing Useful Libraries

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.nn.functional import softmax

# Device Configuration

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Image Preprocessing 
## Normalizing Transforms

In [ ]:
normalize_transform = transforms.Normalize(
        mean=torch.tensor([125.29552899, 122.99125831, 113.90624687]) / 256,
        std=torch.tensor([62.9836127, 62.04402182, 66.63918649]) / 256
    )
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
    normalize_transform])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize_transform])

# Datasets and DataLoader
### Default setup on CIFAR10 dataset, for running on CIFAR100 change CIFAR10 to CIFAR100 on train_dataset and test_dataset

In [ ]:

train_dataset = torchvision.datasets.CIFAR100(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR100(root='../../data/',
                                            train=False, 
                                            transform=transform_test)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=128, 
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=128, 
                                          shuffle=False)

# Defining Resnet Structure

In [ ]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels, track_running_stats=False)
        self.relu = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(p=0.5)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels, track_running_stats=False)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16, track_running_stats=False)
        self.relu = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc = nn.Linear(64, num_classes, bias = True)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels, track_running_stats=False))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
            out = self.conv(x)
            out = self.bn(out)
            out = self.relu(out)
            out = self.dropout1(out)
            out = self.layer1(out)
            out = self.layer2(out)
            out = self.layer3(out)
            out = self.avg_pool(out)
            out = out.view(out.size(0), -1)
            out = self.dropout2(out)
            out = self.fc(out)
            return out

# Hyperparameters

In [ ]:
lr = 0.1
momentum = 0.9
weight_decay = 0.005
num_epochs = 200
n = 125

# Defining Resnet20 baseline

In [ ]:
model = ResNet(ResidualBlock, [3, 3, 3],num_classes=100).to(device)

# Defining Loss and Learning Rate Schedulers
### defined nll loss and cosine learning rate scheduler

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.nn import NLLLoss

# Loss and optimizer
criterion = NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr,momentum = 0.9,weight_decay=0.005)
scheduler = CosineAnnealingWarmRestarts(optimizer,T_0 = num_epochs,eta_min = 0)

# Standard Training

In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = softmax(model(images))
        loss = criterion(torch.log(outputs+ 1e-015), labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    scheduler.step()

# Majority Training

In [ ]:
total_step = len(train_loader)
curr_lr = lr
n = 25
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = torch.zeros(len(images),10).to(device)
        for _ in range(n):
            outputs += softmax(model(images))
        outputs = outputs/n + 1e-015
        loss = criterion(torch.log(outputs), labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    scheduler.step()

# Standard Test

In [ ]:
for i in range(3):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = softmax(model(images)) 
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Mean test

In [ ]:
for i in range(3):
    model.train()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            # results = torch.zeros()
            results = torch.empty(n,len(images)).to(device)
            for i in range(n):
                outputs = softmax(model(images))
                results[i] = torch.max(outputs.data, 1)[1]
            predicted, _ = torch.mode(results, dim=0)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Majority Test

In [ ]:
for i in range(3):
    model.train()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = torch.zeros(len(images),10).to(device)
            for _ in range(n):
                outputs += softmax(model(images))
            outputs /= n     
            _,predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))